In [1]:
import pandas as pd
import numpy as np
import ast
from pymongo import MongoClient
from dotenv import load_dotenv
import os
import re
load_dotenv()
import requests
from bs4 import BeautifulSoup

In [2]:
df=pd.read_csv('penguins_size.csv')

In [3]:
df

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


In [4]:
#sustitucion nan de la columna sex por '0'
sustitucion=df['sex'].fillna('0',inplace=True)

In [5]:
df['sex'].unique()

array(['MALE', 'FEMALE', '0', '.'], dtype=object)

In [6]:
#extraccion de los indices para eliminarlos
df.loc[df['sex'] == '0'].index.tolist()

[3, 8, 9, 10, 11, 47, 246, 286, 324, 339]

In [7]:
#borrado de los registros donde el sexo es 0(nan)
df_clean=df.drop(df.loc[df['sex'] == '0'].index.tolist())

In [8]:
df_clean.head(5)

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE


In [9]:
df_clean['sex'].unique()

array(['MALE', 'FEMALE', '.'], dtype=object)

In [10]:
df.loc[df['sex'] == '.']#replace . por female

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
336,Gentoo,Biscoe,44.5,15.7,217.0,4875.0,.


In [11]:
df_clean.replace({'.':'FEMALE'},inplace=True)

In [12]:
df_clean['sex'].unique()

array(['MALE', 'FEMALE'], dtype=object)

In [13]:
df.loc[df['species'] == 'Gentoo']

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
220,Gentoo,Biscoe,46.1,13.2,211.0,4500.0,FEMALE
221,Gentoo,Biscoe,50.0,16.3,230.0,5700.0,MALE
222,Gentoo,Biscoe,48.7,14.1,210.0,4450.0,FEMALE
223,Gentoo,Biscoe,50.0,15.2,218.0,5700.0,MALE
224,Gentoo,Biscoe,47.6,14.5,215.0,5400.0,MALE
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,0
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


In [14]:
df[df.species.isin(['Gentoo']) & df.sex.isin(['FEMALE'])].describe()


,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
count,58.000000,58.000000,58.000000,58.000000
mean,45.563793,14.237931,212.706897,4679.741379
std,2.051247,0.540249,3.897856,281.578294
min,40.900000,13.100000,203.000000,3950.000000
25%,43.850000,13.800000,210.000000,4462.500000
50%,45.500000,14.250000,212.000000,4700.000000
75%,46.875000,14.600000,215.000000,4875.000000
max,50.500000,15.500000,222.000000,5200.000000


In [15]:
df[df.species.isin(['Gentoo']) & df.sex.isin(['MALE'])].describe()

,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
count,61.000000,61.000000,61.000000,61.000000
mean,49.473770,15.718033,221.540984,5484.836066
std,2.720594,0.741060,5.673252,313.158596
min,44.400000,14.100000,208.000000,4750.000000
25%,48.100000,15.200000,218.000000,5300.000000
50%,49.500000,15.700000,221.000000,5500.000000
75%,50.500000,16.100000,225.000000,5700.000000
max,59.600000,17.300000,231.000000,6300.000000


In [16]:
df_clean.describe(include='all')

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
count,334,334,334.000000,334.000000,334.000000,334.000000,334
unique,3,3,NaN,NaN,NaN,NaN,2
top,Adelie,Biscoe,NaN,NaN,NaN,NaN,MALE
freq,146,164,NaN,NaN,NaN,NaN,168
mean,NaN,NaN,43.994311,17.160479,201.014970,4209.056886,NaN
std,NaN,NaN,5.460521,1.967909,14.022175,804.836129,NaN
min,NaN,NaN,32.100000,13.100000,172.000000,2700.000000,NaN
25%,NaN,NaN,39.500000,15.600000,190.000000,3550.000000,NaN
50%,NaN,NaN,44.500000,17.300000,197.000000,4050.000000,NaN
75%,NaN,NaN,48.575000,18.700000,213.000000,4793.750000,NaN


In [17]:
df_clean['island'].unique()

array(['Torgersen', 'Biscoe', 'Dream'], dtype=object)

    ##coordenadas##


In [18]:
#Isla dream

In [19]:
resD = requests.get("https://es.wikipedia.org/wiki/Isla_Dream")
resD

<Response [200]>

In [20]:
soup=BeautifulSoup(resD.text)

In [21]:
for latitud_isD in soup.select('*.latitude'):
    lat_ISD=latitud_isD.text.rstrip(', ')

In [22]:
lat_ISD

'-64.733'

In [23]:
for longitud_isD in soup.select('*.longitude'):
    lon_ISD=longitud_isD.text.rstrip(', ')

In [24]:
lon_ISD

'-64.233'

In [25]:
#Isla Biscoe

In [26]:
resB = requests.get('https://es.wikipedia.org/wiki/Islas_Biscoe')
resB

<Response [200]>

In [27]:
soup=BeautifulSoup(resB.text)

In [28]:
for longitud_isB in soup.select('*.longitude'):
    lon_ISB=longitud_isB.text
lon_ISB

'-65.500'

In [29]:
for latitud_isB in soup.select('*.latitude'):
    lat_ISB=latitud_isB.text.rstrip(', ')
lat_ISB

'-65.433'

In [30]:
#Isla Torgersen

In [31]:
resT = requests.get('https://es.wikipedia.org/wiki/Isla_Torgersen')
resT 

<Response [200]>

In [32]:
soup=BeautifulSoup(resT.text)

In [33]:
for longitud_isT in soup.select('*.longitude'):
    lon_IST=longitud_isT.text
lon_IST

'-64.083'

In [34]:
for latitud_isT in soup.select('*.latitude'):
    lat_IST=latitud_isT.text.rstrip(', ')
lat_IST

'-64.767'

In [35]:
def set_values(row, value):
    return value[row]   
map_dictionary ={'Biscoe': lon_ISB, 'Dream':lon_ISD, 'Torgersen':lon_IST}   
df_clean['coord_longitud'] = df_clean['island'].apply(set_values, args =(map_dictionary, ))


In [36]:
def set_values(row, value):
    return value[row]   
map_dictionary ={'Biscoe': lat_ISB, 'Dream':lat_ISD, 'Torgersen':lat_IST}   
df_clean['coord_latitud'] = df_clean['island'].apply(set_values, args =(map_dictionary, ))


In [37]:
df_clean

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex,coord_longitud,coord_latitud
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE,-64.083,-64.767
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE,-64.083,-64.767
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE,-64.083,-64.767
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE,-64.083,-64.767
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE,-64.083,-64.767
...,...,...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,FEMALE,-65.500,-65.433
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE,-65.500,-65.433
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE,-65.500,-65.433
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE,-65.500,-65.433


In [38]:
#convertir la bbdd a diccionario
bbdd_Mongo=df_clean.to_dict('index')

In [39]:
len(bbdd_Mongo)

334

In [40]:
#convertir diccionario a una lista
lista_bbdd=[]
for y in bbdd_Mongo.values():
    lista_bbdd.append(y)

In [41]:
lista_bbdd

[{'species': 'Adelie',
  'island': 'Torgersen',
  'culmen_length_mm': 39.1,
  'culmen_depth_mm': 18.7,
  'flipper_length_mm': 181.0,
  'body_mass_g': 3750.0,
  'sex': 'MALE',
  'coord_longitud': '-64.083',
  'coord_latitud': '-64.767'},
 {'species': 'Adelie',
  'island': 'Torgersen',
  'culmen_length_mm': 39.5,
  'culmen_depth_mm': 17.4,
  'flipper_length_mm': 186.0,
  'body_mass_g': 3800.0,
  'sex': 'FEMALE',
  'coord_longitud': '-64.083',
  'coord_latitud': '-64.767'},
 {'species': 'Adelie',
  'island': 'Torgersen',
  'culmen_length_mm': 40.3,
  'culmen_depth_mm': 18.0,
  'flipper_length_mm': 195.0,
  'body_mass_g': 3250.0,
  'sex': 'FEMALE',
  'coord_longitud': '-64.083',
  'coord_latitud': '-64.767'},
 {'species': 'Adelie',
  'island': 'Torgersen',
  'culmen_length_mm': 36.7,
  'culmen_depth_mm': 19.3,
  'flipper_length_mm': 193.0,
  'body_mass_g': 3450.0,
  'sex': 'FEMALE',
  'coord_longitud': '-64.083',
  'coord_latitud': '-64.767'},
 {'species': 'Adelie',
  'island': 'Torgersen'

In [42]:
geo_data=[]
for loc in lista_bbdd:
    geojson={
        'type':'Point',
        "coordinates":[float(loc['coord_latitud']),float(loc['coord_longitud'])]
    }
    
    loc['location']=geojson
    loc.pop('coord_longitud')
    loc.pop('coord_latitud')
    geo_data.append(loc)
print(geo_data)



[{'species': 'Adelie', 'island': 'Torgersen', 'culmen_length_mm': 39.1, 'culmen_depth_mm': 18.7, 'flipper_length_mm': 181.0, 'body_mass_g': 3750.0, 'sex': 'MALE', 'location': {'type': 'Point', 'coordinates': [-64.767, -64.083]}}, {'species': 'Adelie', 'island': 'Torgersen', 'culmen_length_mm': 39.5, 'culmen_depth_mm': 17.4, 'flipper_length_mm': 186.0, 'body_mass_g': 3800.0, 'sex': 'FEMALE', 'location': {'type': 'Point', 'coordinates': [-64.767, -64.083]}}, {'species': 'Adelie', 'island': 'Torgersen', 'culmen_length_mm': 40.3, 'culmen_depth_mm': 18.0, 'flipper_length_mm': 195.0, 'body_mass_g': 3250.0, 'sex': 'FEMALE', 'location': {'type': 'Point', 'coordinates': [-64.767, -64.083]}}, {'species': 'Adelie', 'island': 'Torgersen', 'culmen_length_mm': 36.7, 'culmen_depth_mm': 19.3, 'flipper_length_mm': 193.0, 'body_mass_g': 3450.0, 'sex': 'FEMALE', 'location': {'type': 'Point', 'coordinates': [-64.767, -64.083]}}, {'species': 'Adelie', 'island': 'Torgersen', 'culmen_length_mm': 39.3, 'culme

In [43]:
#establecer conexion con MongoDB

url=os.getenv("url")
client = MongoClient(url)
client

MongoClient(host=['prueba-shard-00-01.bwtyc.mongodb.net:27017', 'prueba-shard-00-00.bwtyc.mongodb.net:27017', 'prueba-shard-00-02.bwtyc.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-1281do-shard-0', ssl=True)

In [44]:
#establecer conexion con Collectiom Prueba
bdmp=client.get_database('Prueba')

In [45]:
#subir a mongo la lista de datos
bdmp.penguins.insert_many(geo_data)